<a href="https://colab.research.google.com/github/Raysk1/machineLearning/blob/main/PROYECTO_03_FRANCO_GARCIA_OSCAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importando librerias

In [1]:
!pip install pyngrok
!pip install ngrok
!pip install uvicorn
!pip install nest-asyncio
!pip install FastAPI

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 745 kB 24.4 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=089ffcaa82a9bc1aa9e7419dde56dc5c4c386d4e351325b04d52987ca6033c45
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for ngrok: filename=ngrok-0.0.1-py3-none-any.whl size=3061 sha256=b25c17d652d0114ef43f24c0a994dbfebf39ee02d33be1daf486c936dd96b8fc
  Stored in directory: /root/.cache/pip/wheels/b2/f2/1e/11e57b35a7c1ea622a4cff4d9622bebd373f6371f3b1728a00
Successfully built ngrok
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 57 kB 3.1 MB/s 
     |██████████████████

In [2]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split 
from sklearn import metrics 
import pickle
from fastapi import FastAPI
import uvicorn
import nest_asyncio
from pyngrok import ngrok

#Comprobando el dataset

In [3]:
#cargando el dataset
df = pd.read_csv("/content/cleaned.csv")
df.head()

,track_id,genre_top,acousticness,danceability,energy,instrumentalness,liveness,speechiness,tempo,valence
0,153,Rock,0.988306,0.255661,0.979774,0.973006,0.121342,0.051740,90.241,0.034018
1,154,Rock,0.970135,0.352946,0.023852,0.957113,0.113261,0.032177,53.758,0.035632
2,155,Rock,0.981657,0.142249,0.912122,0.967294,0.363510,0.087527,91.912,0.034325
3,169,Rock,0.989141,0.225978,0.722835,0.263076,0.092371,0.053406,94.322,0.028347
4,170,Rock,0.886660,0.298518,0.744333,0.920950,0.139587,0.088781,97.880,0.073548


In [4]:
#Numero de filas y columnas
print("columnas = ", df.shape[1])
print("filas = ", df.shape[0])

columnas =  10
filas =  4802


In [5]:
#Calculando la media, mediana, maximo, minimo, etc
df.describe()

,track_id,acousticness,danceability,energy,instrumentalness,liveness,speechiness,tempo,valence
count,4802.000000,4.802000e+03,4802.000000,4802.000000,4802.000000,4802.000000,4802.000000,4802.000000,4802.000000
mean,30164.871720,4.870600e-01,0.436556,0.625126,0.604096,0.187997,0.104877,126.687944,0.453413
std,28592.013796,3.681396e-01,0.183502,0.244051,0.376487,0.150562,0.145934,34.002473,0.266632
min,2.000000,9.491000e-07,0.051307,0.000279,0.000000,0.025297,0.023234,29.093000,0.014392
25%,7494.250000,8.351236e-02,0.296047,0.450757,0.164972,0.104052,0.036897,98.000750,0.224617
50%,20723.500000,5.156888e-01,0.419447,0.648374,0.808752,0.123080,0.049594,124.625500,0.446240
75%,44240.750000,8.555765e-01,0.565339,0.837016,0.915472,0.215151,0.088290,151.450000,0.666914
max,124722.000000,9.957965e-01,0.961871,0.999768,0.993134,0.971392,0.966177,250.059000,0.983649


#Entrenando al modelo

In [6]:
#dividiendo el dataset en features y variable objetivo
feature_cols = ["acousticness","danceability","energy","instrumentalness","liveness","speechiness","tempo","valence"]
X = df[feature_cols] # Features
y = df.genre_top # variable objetivo

In [7]:
#Dividiendo el dateaset en entrenamiento y pruebas
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) # 70% entrenamiento 30% pruebas

In [8]:
#Configurando el modelo
dtc = DecisionTreeClassifier(criterion="entropy", max_depth=3)

# entrenando el Decision Tree Classifer
dtc = dtc.fit(X_train.values,y_train.values)

#Calculando las predicciones
y_pred = dtc.predict(X_test.values)

In [9]:
#Presicion del modelo
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.9000693962526024


#Exportacion del modelo

In [10]:
#Exportando el modelo con pickle
pickle.dump(dtc, open("Genre_predict.pkl","wb"))


#Creando la clase para las predicciones

In [11]:
from pydantic import BaseModel

class Music(BaseModel):
    acousticness : float 
    danceability : float
    energy : float
    instrumentalness : float
    liveness : float
    speechiness : float
    tempo : float
    valence : float
    class Config:
        schema_extra = {   #EJEMPLO DE LOS DATOS QUE SE LE INSERTARÍAN
            "example": {
                "acousticness" : 0.988306,
                "danceability" : 0.255661,
                "energy" : 0.979774,
                "instrumentalness" : 0.973006,
                "liveness" : 0.121342,
                "speechiness" : 0.051740,
                "tempo" : 90.241,
                "valence" : 0.034018
            }
        }

#Configurando la API

In [12]:
#creando la API
app = FastAPI()

@app.on_event("startup") #Funcion que se ejecuta en el arranque de la API
def load_model():#Carga el modelo
  global model
  #cargando modelo
  model = pickle.load(open("Genre_predict.pkl","rb"))

#Mensaje que indica que la API inicio exitosamente
@app.get("/")
def index():
  return {"PUNTO DE ACCESO API"}

#Funcion para crear la prediccion
@app.post('/predict')
def get_music_category(data: Music):
  received = data.dict()
  acousticness = received['acousticness']
  danceability = received['danceability']
  energy = received['energy']
  instrumentalness = received['instrumentalness']
  liveness = received['liveness']
  speechiness = received['speechiness']
  tempo = received['tempo']
  valence = received['valence']
  #Creando la prediccion  
  prediction = model.predict([[acousticness,danceability,energy,instrumentalness,liveness,speechiness,tempo,valence]]).tolist()[0]
  return {'prediction': prediction} #Y regresa esa predicción.
   
    

In [13]:
! ngrok authtoken 2DGCG0eBlOi7BPf6BRkDaEbhXVl_3c2WJ72qywkPtQGQfCkGK #PONER TU AUTHTOKEN.

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [14]:
#Iniciando la Api
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

INFO:     Started server process [60]
INFO:uvicorn.error:Started server process [60]
INFO:     Waiting for application startup.
INFO:uvicorn.error:Waiting for application startup.
INFO:     Application startup complete.
INFO:uvicorn.error:Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:uvicorn.error:Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: http://20ba-34-86-224-90.ngrok.io
INFO:     200.68.151.21:0 - "GET / HTTP/1.1" 200 OK
INFO:     200.68.151.21:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     200.68.151.21:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     200.68.151.21:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     200.68.151.21:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     200.68.151.21:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     200.68.151.21:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     200.68.151.21:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     200.68.151.21:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     200.68.151.21:0 - "POST /predict HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:uvicorn.error:Shutting down
INFO:     Waiting for application shutdown.
INFO:uvicorn.error:Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:uvicorn.error:Application shutdown complete.
INFO:     Finished server process [60]
INFO:uvicorn.error:Finished server process [60]


In [16]:
ngrok.kill()